In [36]:
import sqlite3
def list_tables(database_file):
    connection = sqlite3.connect('data.db')
    cursor = connection.cursor()
    cursor.execute("""select name from sqlite_master where type = 'table';
                    """)
                    
    tables = cursor.fetchall()
    cursor.close()
    connection.close()
    table_names = [table[0] for table in tables]
    return table_names

In [37]:
import pandas as pd
connection = sqlite3.connect('data.db')
query = """select *
        from client as cl;
        """
client = pd.read_sql_query(query, connection)
connection.close

<function Connection.close>

In [38]:
import pandas as pd
connection = sqlite3.connect('data.db')
query = """select *
        from client_products as cl;
        """
client_products = pd.read_sql_query(query, connection)
connection.close

<function Connection.close>

In [39]:
import pandas as pd
connection = sqlite3.connect('data.db')
query = """select *
        from balances as cl;
        """
balances = pd.read_sql_query(query, connection)
connection.close

<function Connection.close>

In [40]:
import pandas as pd
connection = sqlite3.connect('data.db')
query = """select *
        from inv_campaign_eval as cl;
        """
inv_campaign_eval = pd.read_sql_query(query, connection)
connection.close

<function Connection.close>

In [41]:
connection = sqlite3.connect('data.db')
query = """
        select *
        from client as cl
        left join client_products as cp on cl.client_id = cp.client_id
        --left join balances as ba on cl.client_id = ba.client_id
        left join inv_campaign_eval as camp on cl.client_id = camp.client_id;
        """
clients_merged = pd.read_sql_query(query, connection)
connection.close

<function Connection.close>

In [42]:
my_database_file_path = "/content/data.db"

In [43]:
print(client.head(10))
print(client_products.head(10))
print(balances.head(10))
print(inv_campaign_eval.head(10))
print(clients_merged.head(10))

    client_id   age            job   marital  education gender
0   249789938  38.0       services   married  secondary      M
1  1222646323  46.0       services  divorced    unknown      M
2   451375919  33.0         admin.    single  secondary      F
3   338972671  44.0  self-employed   married  secondary      F
4  1472834688  36.0    blue-collar   married    primary      M
5  1068680340  63.0         admin.   married  secondary      F
6   711558356  38.0  self-employed    single   tertiary      F
7   422084751  30.0     technician    single  secondary      M
8   123316323  36.0    blue-collar  divorced  secondary      F
9  1504633819  38.0     management   married   tertiary      M
    client_id has_deposits loan has_insurance has_mortgage
0   249789938          yes   no             n           no
1  1222646323           no   no             n           no
2   451375919           no   no             n          yes
3   338972671           no   no             y          yes
4  147283468

In [44]:
# Corrected filter
filter_condition = (clients_merged["poutcome"] == 'failure')

# Add a new column "has_family" and initialize it to 0
clients_merged["success"] = 1

# Set "has_family" to 1 where the filter condition is met
clients_merged.loc[filter_condition, "success"] = 0

# Display the DataFrame
print(clients_merged.head())

    client_id   age            job   marital  education gender   client_id  \
0   249789938  38.0       services   married  secondary      M   249789938   
1  1222646323  46.0       services  divorced    unknown      M  1222646323   
2   451375919  33.0         admin.    single  secondary      F   451375919   
3   338972671  44.0  self-employed   married  secondary      F   338972671   
4  1472834688  36.0    blue-collar   married    primary      M  1472834688   

  has_deposits loan has_insurance has_mortgage    client_id poutcome  success  
0          yes   no             n           no  249789938.0  success        1  
1           no   no             n           no          NaN     None        1  
2           no   no             n          yes          NaN     None        1  
3           no   no             y          yes          NaN     None        1  
4          yes   no             n          yes          NaN     None        1  


In [45]:
clients_merged.isnull().sum()

client_id           0
age               232
job               438
marital             0
education           0
gender              0
client_id           0
has_deposits        0
loan                0
has_insurance       0
has_mortgage        0
client_id        8863
poutcome         8863
success             0
dtype: int64

In [46]:
clients_merged.dtypes

client_id          int64
age              float64
job               object
marital           object
education         object
gender            object
client_id          int64
has_deposits      object
loan              object
has_insurance     object
has_mortgage      object
client_id        float64
poutcome          object
success            int64
dtype: object

In [47]:
clients_merged['job'].fillna('unknown', inplace=True)

# Display the updated DataFrame
print(client.head())

    client_id   age            job   marital  education gender
0   249789938  38.0       services   married  secondary      M
1  1222646323  46.0       services  divorced    unknown      M
2   451375919  33.0         admin.    single  secondary      F
3   338972671  44.0  self-employed   married  secondary      F
4  1472834688  36.0    blue-collar   married    primary      M


In [28]:
client.isnull().sum()

client_id      0
age          232
job          438
marital        0
education      0
gender         0
dtype: int64

In [29]:
connection = sqlite3.connect('data.db')
query = """
        select *
        from client as cl
        left join client_products as cp on cl.client_id = cp.client_id;
        """
client_and_products = pd.read_sql_query(query, connection)
connection.close

client_and_products = client_and_products.T.drop_duplicates().T
client_and_products.head()
client_and_products.dtypes


client_id        object
age              object
job              object
marital          object
education        object
gender           object
has_deposits     object
loan             object
has_insurance    object
has_mortgage     object
dtype: object

In [35]:
connection = sqlite3.connect('data.db')
query = """
        select *
        from client as cl
        left join client_products as cp on cl.client_id = cp.client_id;
        """
client_and_products = pd.read_sql_query(query, connection)
connection.close

client_and_products = client_and_products.T.drop_duplicates().T
client_and_products.head()
client_and_products.dtypes

client_id        object
age              object
job              object
marital          object
education        object
gender           object
has_deposits     object
loan             object
has_insurance    object
has_mortgage     object
dtype: object

In [55]:
# Assuming df is your DataFrame with columns 'client_id' and 'timestamp'
# Make sure 'timestamp' is in datetime format
balances['date'] = pd.to_datetime(balances['date'])

# Get the index of the last timestamp for each client_id
last_timestamp_index = balances.groupby('client_id')['date'].idxmax()

# Use the index to extract the corresponding rows from the original DataFrame
last_timestamp_df = balances.loc[last_timestamp_index]

# Display the DataFrame with the last timestamp for each client_id
print(last_timestamp_df)



# Convert 'datetime_column' to datetime format
balances['date'] = pd.to_datetime(balances['date'])

# Extract just the date from the datetime column
balances['date'] = balances['date'].dt.date

print(balances)

balances = pd.DataFrame(balances)

# Convert 'datetime_column' to datetime format
balances['date'] = pd.to_datetime(balances['date'])

date_frequency = balances.groupby(balances['date'].dt.date).size().reset_index(name='frequency')

# Display the resulting DataFrame with date frequencies
print(date_frequency)



             date      balance   client_id currency
265876 2023-06-06   175.445269   100070076      CZK
175292 2023-06-06  2559.041192   100141910      CZK
191620 2023-06-06    94.055761   100207570      CZK
43680  2023-06-06   382.755389   100488560      CZK
262678 2023-06-06   369.476219   100605013      CZK
...           ...          ...         ...      ...
22854  2023-06-06  1146.101785  1999463182      CZK
130338 2023-06-06    30.830676  1999499145      CZK
87074  2023-06-06   792.767525  1999715381      CZK
89154  2023-06-06  5077.753230  1999945413      CZK
232674 2023-06-06   229.440915  1999956516      CZK

[11162 rows x 4 columns]
              date      balance  client_id currency
0       2023-06-06  2680.467773  249789938      CZK
1       2023-05-23  2654.055419  249789938      CZK
2       2023-05-09  2620.835872  249789938      CZK
3       2023-04-25  2720.315925  249789938      CZK
4       2023-04-11  2687.601982  249789938      CZK
...            ...          ...       

In [56]:
print(date_frequency)

          date  frequency
0   2022-06-21      11162
1   2022-07-05      11162
2   2022-07-19      11162
3   2022-08-02      11162
4   2022-08-16      11162
5   2022-08-30      11162
6   2022-09-13      11162
7   2022-09-27      11162
8   2022-10-11      11162
9   2022-10-25      11162
10  2022-11-08      11162
11  2022-11-22      11162
12  2022-12-06      11162
13  2022-12-20      11162
14  2023-01-03      11162
15  2023-01-17      11162
16  2023-01-31      11162
17  2023-02-14      11162
18  2023-02-28      11162
19  2023-03-14      11162
20  2023-03-28      11162
21  2023-04-11      11162
22  2023-04-25      11162
23  2023-05-09      11162
24  2023-05-23      11162
25  2023-06-06      11162


In [57]:
balances_mean = pd.DataFrame(balances)

# Convert 'date' to datetime format
balances['date'] = pd.to_datetime(balances['date'])

# Group by 'client_id' and calculate the mean balance for each client
mean_balance_by_client = balances.groupby('client_id')['balance'].mean().reset_index()

# Display the resulting DataFrame with mean balances
print(mean_balance_by_client)

        client_id      balance
0       100070076   203.907928
1       100141910  2544.965438
2       100207570   -15.679495
3       100488560   381.488613
4       100605013   420.351449
...           ...          ...
11157  1999463182  1268.089349
11158  1999499145    -4.751058
11159  1999715381   676.988885
11160  1999945413  5047.496305
11161  1999956516   231.982319

[11162 rows x 2 columns]
